# Alle Karteringen omzetten

Dit notebook voert alle karteringen via veg2hab naar een habitattypekartering. Instructies:

- Kopieer de provinciefolders uit de OneDrive: `Spheer AI/Spheer AI - General/Projecten/Veg_2_Hab/Data/Vegetatiekarteringen` naar `testing/vegetatiekarteringen` (of naar iets anders, dan moet je `originele_karteringen_prefix` aanpassen onder **config**)
- Eerst moeten sommige shapefiles opgeschoond worden. Dit kan met het `handmatig_opschonen_shapefiles.ipynb` notebook. Run deze als dat niet al eens gedaan is.
- Zorg dat de paden `originele_karteringen_prefix` en `opgeschoonde_shapefiles_prefix` kloppen en overeenkomen met dezelfde paden in het `handmatig_opschonen_shapefiles.ipynb`. Deze prefixes zullen voor de paden vermeld in de spreadsheets geplakt worden.
- Run de cellen. Deze zullen:
  - Imports importen en het bestaan van de paden controleren. 
  - De voor het omzetten benodigde klassen instantieren; de was-wordt lijst, de definitietabel en de FGR kaart.
  - Spreadsheets met daarin configuraties voor iedere kartering inlezen.
  - Alle karteringen inladen in een grote dict.
  - De volgende stappen een voor een op iedere kartering uitvoeren:
    - De was-wordt lijst toepassen (toevoegen van VvN codes aan karteringen met enkel SBB).
    - De definitietabel toepassen (het opstellen van habitatvoorstellen voor ieder vlak).
    - De mitsen checken (eventuele criteria in de habitatvoorstellen checken) en voor ieder vegetatietype een habitatkeuze maken.
    - De kartering formatten als een habitattypekartering en wegschrijven.  

### Imports etc

In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.fgr import FGR

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 8)

originele_karteringen_prefix = '../testing/vegetatiekarteringen/'
opgeschoonde_shapefiles_prefix = '../testing/opgeschoonde_shapefiles/'
output_prefix = '../testing/omgezette_vegetatiekarteringen/'

assert Path(originele_karteringen_prefix).exists(), \
    f"De map {originele_karteringen_prefix} bestaat niet. Zet hier de vegetatiekarteringen uit de onedrive in."

assert Path(opgeschoonde_shapefiles_prefix).exists(), \
    f"De map {opgeschoonde_shapefiles_prefix} bestaat niet. Maak deze eerst aan met het handmatig_opschonen_shapefiles.ipynb notebook."


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Omzetclasses instantieren

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)

path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.csv")
path_in_mozaiekjson = Path("../data/mozaiekjson.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)

fgr = FGR(Path("../data/bronbestanden/FGR.json"))

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


### Inladen benodigde informatie

**Overzicht_vegetatiekarteringen.xlsx** is uit de drive en weet van karteringen de provincie, naam, shapefile pad en of het een access of een shapefile kartering is.

**accesskarteringinfo.xlsx** bevat voor de access karteringen de informatie die nodig is om een access kartering in te laden.

**shapefilekarteringinfo.xlsx** bevat voor de shapefile karteringen de informatie die nodig is om een shapefile kartering in te laden.


In [3]:
# info over de karteringen
overzicht = pd.read_excel(
    "../data/Overzicht_vegetatiekarteringen.xlsx", 
    sheet_name="Vegetatiekarteringen", 
    engine='openpyxl',
    usecols=[
        "provincie", 
        "naam_kartering", 
        "path_shapes", 
        "landelijke_typologie beschikbaar?", 
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de access karteringen
access_info = pd.read_excel(
    "../data/accesskarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "path_csvs",
        "elm_id_name",
    ],
    dtype="string"
)

# benodigde info voor de shapefile karteringen
shape_info = pd.read_excel(
    "../data/shapefilekarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "ElmID_col",
        "datum_col",
        "opmerking_col",
        "vegtype_col_format",
        "sbb_of_vvn",
        "SBB_col",
        "VvN_col",
        "split_char",
        "perc_col",
    ],
    dtype="string"
)
# Omzetten van pd.NA naar None om te voorkomen dat we overal "if not pd.isna()" moeten doen
# NOTE: dit naar vegkaartering.py?
shape_info = shape_info.applymap(lambda x: None if pd.isna(x) else x)

overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
# omzetten van ./PROVINCIE/gebied/gebied.shp naar ../testing/opgeschoonde_shapefiles/PROVINCIE/gebied/gebied.shp
overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: opgeschoonde_shapefiles_prefix + x[2:])

access_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "access"]
access_overzicht = access_overzicht.join(access_info.set_index("naam_kartering"), on="naam_kartering")
# omzetten van ./PROVINPCIE/gebied/gebied.shp naar ../testing/vegetatiekarteringen/PROVINCIE/gebied/gebied.shp
access_overzicht.path_csvs = access_overzicht.path_csvs.apply(lambda x: originele_karteringen_prefix + x[2:])

shapefile_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "in shapes"]
shapefile_overzicht = shapefile_overzicht.join(shape_info.set_index("naam_kartering"), on="naam_kartering")

In [4]:
shapefile_overzicht

,provincie,naam_kartering,path_shapes,landelijke_typologie beschikbaar?,locatie_landelijke_typologie,ElmID_col,datum_col,opmerking_col,vegtype_col_format,sbb_of_vvn,SBB_col,VvN_col,split_char,perc_col
0,Drenthe,Asserbos2015,../testing/opgeschoonde_shapefiles/DR/Asserbos...,Ja,in shapes,OBJECTID,DATE,None,single,VvN,None,CODE_VvN,None,None
1,Drenthe,BG_Drenthe2014,../testing/opgeschoonde_shapefiles/DR/BG_Drent...,Ja,in shapes,None,JAAR,None,single,beide,CODE_SBB,CODE_VvN,None,None
2,Drenthe,BG_Holtingerveld_bossen2016,../testing/opgeschoonde_shapefiles/DR/BG_Holti...,Ja,in shapes,OBJECTID_1,DATUM,OPMERKING,multi,beide,F1_SBB+F2_SBB+F3_SBB+F4_SBB,F1_VvN+F2_VvN+F3_VvN+F4_VvN,+,F1_Perc+F2_Perc+F3_Perc+F4_Perc
3,Drenthe,BG_Hondsrug2019,../testing/opgeschoonde_shapefiles/DR/BG_Honds...,Ja,in shapes,Id,Datum,None,multi,beide,SBBTYPE1+SBBTYPE2,VvN1+VvN2,+,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Friesland,Tsjonger_Middenloop,../testing/opgeschoonde_shapefiles/FR/Tsjonger...,Ja,in shapes,ELMID,DATUM,OPMERKING,multi,SBB,Sbbcatal1+Sbbcatal2,None,+,None
74,Groningen,SBB ZWK 2010 0814_Tussen de Gasten 2010,../testing/opgeschoonde_shapefiles/GR/SBB ZWK ...,Ja,in shapes,Elmid,Datum,Opmerking,single,SBB,SBBTYPE,None,+,None
76,Groningen,SGL Hunzedal en Leekstermeer2021 Hunzedal,../testing/opgeschoonde_shapefiles/GR/SGL Hunz...,Ja,in shapes,ElmID,Datum,Opmerking,single,SBB,SBBTYPE,None,+,None
77,Groningen,SGL Hunzedal en Leekstermeer2021 Leekstermeer ...,../testing/opgeschoonde_shapefiles/GR/SGL Hunz...,Ja,in shapes,elmid,datum,None,single,SBB,SBBTYPE,None,+,None


### Karteringen inladen

Hier worden, adhv configuraties uit de spreadsheets, alle karteringen ingeladen. Als er een probleem is met een kartering, wordt deze overgeslagen en wordt een melding gegeven.

In [5]:
karteringen = {}
mislukte_karteringen = []

for row in access_overzicht.itertuples():
    print(f"------ Inlezen {row.naam_kartering}")
    try:
        karteringen[row.naam_kartering] = Kartering.from_access_db(Path(row.path_shapes), row.elm_id_name, Path(row.path_csvs))
        
    except Exception as e:
        print(e)
        print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")
        mislukte_karteringen.append(row.naam_kartering)
        continue

    print(f"Gelukt!")

for row in shapefile_overzicht.itertuples():
    print(f"------ Inlezen {row.naam_kartering}")
    try:
        karteringen[row.naam_kartering] = Kartering.from_shapefile(
            Path(row.path_shapes),
            row.ElmID_col,
            vegtype_col_format=row.vegtype_col_format,
            sbb_of_vvn=row.sbb_of_vvn,
            VvN_col=row.VvN_col,
            SBB_col=row.SBB_col, 
            split_char=row.split_char, 
            datum_col=row.datum_col, 
            opmerking_col=row.opmerking_col,
            perc_col=row.perc_col  
        )
    
    except Exception as e:
        print(e)
        print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")
        mislukte_karteringen.append(row.naam_kartering)
        continue

    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen zijn niet ingelezen !!!")
    print(mislukte_karteringen)

------ Inlezen NM_Fochteloerveen_2020
Gelukt!
------ Inlezen NM_DCR_2013
Er zijn 476 vlakken zonder VegTypeInfo in ../testing/opgeschoonde_shapefiles/DR Extra/NM_DCR2013/NM_Fochteloerveen_DCR2013.shp. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [nan, nan, nan, nan, nan]
Gelukt!
------ Inlezen NM_Westerbork2017
Gelukt!
------ Inlezen SBB0817_Grolloerveen2013
Gelukt!
------ Inlezen SBB0863_Elperstroom_Schoonloo2012
Gelukt!
------ Inlezen SBB0936_DrentseAa2015_16


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:539: UserWarning: CRS van ../testing/opgeschoonde_shapefiles/DR/SBB0936_DrentseAa2015_16/SBB0936_DrentseAa2015_16.shp was None en is nu gelezen als EPSG:28992
  warnings.warn(f"CRS van {shape_path} was None en is nu gelezen als EPSG:28992")


Gelukt!
------ Inlezen Fr 0752_Bakkefean_2009


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:539: UserWarning: CRS van ../testing/opgeschoonde_shapefiles/FR/Bakkeveense_duinen/Fr 0752_Bakkefean_2009/Vlakken.shp was None en is nu gelezen als EPSG:28992
  warnings.warn(f"CRS van {shape_path} was None en is nu gelezen als EPSG:28992")


Gelukt!
------ Inlezen Slotplaats_2016
Gelukt!
------ Inlezen Ameland_2010


...

Gelukt!
------ Inlezen Vlieland_2013
Gelukt!
------ Inlezen Lauwersmeer_Fryslan_2015
Gelukt!
------ Inlezen Rottige_Meenthe_Brandemeer_2013
Gelukt!
------ Inlezen Sneekermeergebied_2009_Terkaplester puollen
Gelukt!
------ Inlezen Bakkefean_2022
Gelukt!
------ Inlezen Snitsermar_2021
Gelukt!
------ Inlezen Deelen_2022
Gelukt!
------ Inlezen Tjonger_2021
Er zijn 179 vlakken zonder VegTypeInfo in ../testing/opgeschoonde_shapefiles/FR Extra/Tjonger_2021_SBB/Shapefiles/Vlakken/Vlakken_inclSBBtyp.shp. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [1, 2, 3, 4, 5]
Gelukt!
------ Inlezen Witte en zwarte brekken_2021
Er zijn 16 vlakken zonder VegTypeInfo in ../testing/opgeschoonde_shapefiles/FR Extra/Witte en zwarte brekken Veg&Flora 2021/Shapefiles/Shapefiles/Vlakken.shp. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [6, 19, 20, 33, 34]
Gelukt!
------ Inlezen NM vegetatiekartering RuitenAa2020
Er is geen 1 op 1 relatie tussen Elmid

/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:539: UserWarning: CRS van ../testing/opgeschoonde_shapefiles/DR/NM_Mantingerweiden2010/NM_Mantingerweiden2010.shp was None en is nu gelezen als EPSG:28992
  warnings.warn(f"CRS van {shape_path} was None en is nu gelezen als EPSG:28992")


Gelukt!
------ Inlezen NM_Norg_heiden2020
Gelukt!
------ Inlezen NM_NorgerBeekdal2018
Gelukt!
------ Inlezen Bosgroep_SNLMonitoring_2014
Gelukt!
------ Inlezen NM_SBB_Fochteloerveen_randen2016
Gelukt!
------ Inlezen PRV_Witterdiep2021
Gelukt!
------ Inlezen SBB0756_Drentsche_A2009
Gelukt!
------ Inlezen SBB0757_De_Velden2009
Gelukt!
------ Inlezen SBB0763_Terhorst2009
Gelukt!
------ Inlezen SBB0816_Norg2010
Gelukt!
------ Inlezen SBB0881_SBB_Ruinen_Havelterberg2013
Gelukt!
------ Inlezen SBB0935_DrentsFrieseWold2015
Gelukt!
------ Inlezen SBB0982_Wijster2017


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:539: UserWarning: CRS van ../testing/opgeschoonde_shapefiles/DR/SBB0982_Wijster2017/SBB0982_Wijster_Ruinen_Hooghalen2017.shp was None en is nu gelezen als EPSG:28992
  warnings.warn(f"CRS van {shape_path} was None en is nu gelezen als EPSG:28992")


Gelukt!
------ Inlezen SBB1014_SBB_Kop_van_Drenthe2018
Gelukt!
------ Inlezen SBB1015_ HartvanDrenthe2018
Gelukt!
------ Inlezen SBB1034_Dwingelderveld2017
Gelukt!
------ Inlezen SBB1039_Drenthe2019
Gelukt!
------ Inlezen SBB1471_DrentsFrieseWold2008
Gelukt!
------ Inlezen Drouwenerzand_2020
Gelukt!
------ Inlezen NM_Mantingerbos_2015
Gelukt!
------ Inlezen Mandefjild_2015
Gelukt!
------ Inlezen KetlikerSkar_2016
Gelukt!
------ Inlezen Sneekermeergebied_2009
Gelukt!
------ Inlezen Tsjonger_Middenloop
Gelukt!
------ Inlezen SBB ZWK 2010 0814_Tussen de Gasten 2010
Gelukt!
------ Inlezen SGL Hunzedal en Leekstermeer2021 Hunzedal
Gelukt!
------ Inlezen SGL Hunzedal en Leekstermeer2021 Leekstermeer 2021
Gelukt!
!!! De volgende karteringen zijn niet ingelezen !!!
['NM vegetatiekartering RuitenAa2020', 'NM_Leggelderveld2020']


### Karteringen omzetten

In [6]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Waswordtlijst toepassen {key}")
    try:
        kartering.apply_wwl(wwl)    
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen van de was-wordt-lijst !!!")
    print(mislukte_karteringen)

------ Waswordtlijst toepassen NM_Fochteloerveen_2020
Gelukt!
------ Waswordtlijst toepassen NM_DCR_2013
Gelukt!
------ Waswordtlijst toepassen NM_Westerbork2017
Gelukt!
------ Waswordtlijst toepassen SBB0817_Grolloerveen2013
Gelukt!
------ Waswordtlijst toepassen SBB0863_Elperstroom_Schoonloo2012
Gelukt!
------ Waswordtlijst toepassen SBB0936_DrentseAa2015_16
Gelukt!
------ Waswordtlijst toepassen Fr 0752_Bakkefean_2009
Gelukt!
------ Waswordtlijst toepassen Slotplaats_2016
Gelukt!
------ Waswordtlijst toepassen Ameland_2010
Gelukt!
------ Waswordtlijst toepassen Schiermonnikoog_2015
Gelukt!
------ Waswordtlijst toepassen Schiermonnikoog_2017
Gelukt!
------ Waswordtlijst toepassen Terschelling_2012
Gelukt!
------ Waswordtlijst toepassen Vlieland_2013
Gelukt!
------ Waswordtlijst toepassen Lauwersmeer_Fryslan_2015
Gelukt!
------ Waswordtlijst toepassen Rottige_Meenthe_Brandemeer_2013
Gelukt!
------ Waswordtlijst toepassen Sneekermeergebied_2009_Terkaplester puollen
Gelukt!
------ Waswo

/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:968: UserWarning: Er zijn al VvN aanwezig in de kartering. De was-wordt lijst wordt niet toegepast.
  "Er zijn al VvN aanwezig in de kartering. De was-wordt lijst wordt niet toegepast."


Gelukt!
------ Waswordtlijst toepassen SBB0757_De_Velden2009
Gelukt!
------ Waswordtlijst toepassen SBB0763_Terhorst2009
Gelukt!
------ Waswordtlijst toepassen SBB0816_Norg2010
Gelukt!
------ Waswordtlijst toepassen SBB0881_SBB_Ruinen_Havelterberg2013
Gelukt!
------ Waswordtlijst toepassen SBB0935_DrentsFrieseWold2015
Gelukt!
------ Waswordtlijst toepassen SBB0982_Wijster2017
Gelukt!
------ Waswordtlijst toepassen SBB1014_SBB_Kop_van_Drenthe2018
Gelukt!
------ Waswordtlijst toepassen SBB1015_ HartvanDrenthe2018
Gelukt!
------ Waswordtlijst toepassen SBB1034_Dwingelderveld2017
Gelukt!
------ Waswordtlijst toepassen SBB1039_Drenthe2019
Gelukt!
------ Waswordtlijst toepassen SBB1471_DrentsFrieseWold2008
Gelukt!
------ Waswordtlijst toepassen Drouwenerzand_2020
Gelukt!
------ Waswordtlijst toepassen NM_Mantingerbos_2015
Gelukt!
------ Waswordtlijst toepassen Mandefjild_2015
Gelukt!
------ Waswordtlijst toepassen KetlikerSkar_2016
Gelukt!
------ Waswordtlijst toepassen Sneekermeergebied_200

In [7]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Definitietabel toepassen {key}")
    try:  
        kartering.apply_deftabel(dt)        
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen van de definitietabel!!!")
    print(mislukte_karteringen)

------ Definitietabel toepassen NM_Fochteloerveen_2020
Gelukt!
------ Definitietabel toepassen NM_DCR_2013
Gelukt!
------ Definitietabel toepassen NM_Westerbork2017
Gelukt!
------ Definitietabel toepassen SBB0817_Grolloerveen2013
Gelukt!
------ Definitietabel toepassen SBB0863_Elperstroom_Schoonloo2012
Gelukt!
------ Definitietabel toepassen SBB0936_DrentseAa2015_16
Gelukt!
------ Definitietabel toepassen Fr 0752_Bakkefean_2009
Gelukt!
------ Definitietabel toepassen Slotplaats_2016
Gelukt!
------ Definitietabel toepassen Ameland_2010
Gelukt!
------ Definitietabel toepassen Schiermonnikoog_2015
Gelukt!
------ Definitietabel toepassen Schiermonnikoog_2017
Gelukt!
------ Definitietabel toepassen Terschelling_2012
Gelukt!
------ Definitietabel toepassen Vlieland_2013
Gelukt!
------ Definitietabel toepassen Lauwersmeer_Fryslan_2015
Gelukt!
------ Definitietabel toepassen Rottige_Meenthe_Brandemeer_2013
Gelukt!
------ Definitietabel toepassen Sneekermeergebied_2009_Terkaplester puollen
Gelu

In [8]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Mitsen/Mozaiekregels checken {key}")
    try:
        kartering.bepaal_habitatkeuzes(fgr)    
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen checken van de criteria!!!")
    print(mislukte_karteringen)

------ Mitsen/Mozaiekregels checken NM_Fochteloerveen_2020


Iteratie 0: van 3361 naar 7 keuzes nog te bepalen
Iteratie 1: van 7 naar 0 keuzes nog te bepalen
Gelukt!
------ Mitsen/Mozaiekregels checken NM_DCR_2013
Iteratie 0: van 57 naar 0 keuzes nog te bepalen
Gelukt!
------ Mitsen/Mozaiekregels checken NM_Westerbork2017
Iteratie 0: van 207 naar 7 keuzes nog te bepalen
Iteratie 1: van 7 naar 0 keuzes nog te bepalen
Gelukt!
------ Mitsen/Mozaiekregels checken SBB0817_Grolloerveen2013
Iteratie 0: van 2359 naar 33 keuzes nog te bepalen
Iteratie 1: van 33 naar 0 keuzes nog te bepalen
Gelukt!
------ Mitsen/Mozaiekregels checken SBB0863_Elperstroom_Schoonloo2012
Iteratie 0: van 1203 naar 28 keuzes nog te bepalen
Iteratie 1: van 28 naar 0 keuzes nog te bepalen
Gelukt!
------ Mitsen/Mozaiekregels checken SBB0936_DrentseAa2015_16
Iteratie 0: van 21463 naar 972 keuzes nog te bepalen
Iteratie 1: van 972 naar 4 keuzes nog te bepalen
Iteratie 2: van 4 naar 0 keuzes nog te bepalen
Gelukt!
------ Mitsen/Mozaiekregels checken Fr 0752_Bakkefean_2009
Iteratie 0:

### Karteringen wegschrijven

In [9]:
for gebied in (karteringen.keys()):
    print(gebied)
    # Vervangen van de shapefile path naar de output path
    p = Path(overzicht[overzicht.naam_kartering == gebied].path_shapes.str.replace(opgeschoonde_shapefiles_prefix, output_prefix).iloc[0])
    p.parent.mkdir(parents=True, exist_ok=True)
    karteringen[gebied].final_format_to_file(p)

NM_Fochteloerveen_2020


NM_DCR_2013
NM_Westerbork2017
SBB0817_Grolloerveen2013
SBB0863_Elperstroom_Schoonloo2012
SBB0936_DrentseAa2015_16


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1243: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final.to_file(path)


Fr 0752_Bakkefean_2009
Slotplaats_2016
Ameland_2010
Schiermonnikoog_2015
Schiermonnikoog_2017
Terschelling_2012
Vlieland_2013
Lauwersmeer_Fryslan_2015
Rottige_Meenthe_Brandemeer_2013
Sneekermeergebied_2009_Terkaplester puollen
Bakkefean_2022
Snitsermar_2021
Deelen_2022
Tjonger_2021
Witte en zwarte brekken_2021
SBB Duurswold2013
SBB Lauwersmeer 2015
SBB Westerwolde 2020
SBB ZWK 2010 0815_Trimunt_2010
SGL Zuidlaardermeer 2019
Asserbos2015
BG_Drenthe2014
BG_Holtingerveld_bossen2016
BG_Hondsrug2019
BG_meerdere_terreinen2020
BG_Nolde2018
DEF_Holtingerveld2013
GL_Zuidlaardermeer2019
HDL_De_Stroeten2016
HDL_Hijkerveld-Smilder_oosterveld2018
HDL_Katteveen2019
HDL_meerdere_terreinen2016
HDL_meerdere_terreinen2017
IFG_DrentsFrieseWold2015
NM_Fochteloerveen2013_14
NM_Klencke2018
NM_Mantingerveld2020
NM_Mantingerweiden2010
NM_Norg_heiden2020
NM_NorgerBeekdal2018
Bosgroep_SNLMonitoring_2014
NM_SBB_Fochteloerveen_randen2016
PRV_Witterdiep2021
SBB0756_Drentsche_A2009
SBB0757_De_Velden2009
SBB0763_Ter

In [12]:
# tel aantal vlakken
n_vlakken = 0
for kartering in karteringen.values():
    n_vlakken += kartering.gdf.shape[0]

# tel aantal keuzes
n_keuzes = 0
for kartering in karteringen.values():
    n_keuzes += kartering.gdf.HabitatKeuze.apply(len).sum()

print(f"Er zijn {n_vlakken} vlakken en {n_keuzes} keuzes omgezet.")

Er zijn 79669 vlakken en 142828 keuzes omgezet.


In [14]:
# Gemeten op 15-04-2024
secs = 1048

vlakken_per_seconde = n_vlakken / secs
keuzes_per_seconde = n_keuzes / secs

print(f"Dat is {vlakken_per_seconde:.2f} vlakken en {keuzes_per_seconde:.2f} keuzes per seconde.")

seconden_per_kartering = secs / len(karteringen)

print(f"Dat is {seconden_per_kartering:.2f} seconden per kartering.")

Dat is 76.02 vlakken en 136.29 keuzes per seconde.
Dat is 14.97 seconden per kartering.
